In [1]:
import cv2 as cv
import numpy as np

In [2]:
def showimage(image,label = "image"):
    cv.imshow(label,image)
    cv.waitKey()
    cv.destroyAllWindows()

In [3]:
def find_slope(line):
    x1,y1,x2,y2 = line[0]
    return (y2-y1)/(x2-x1)
def length_of_line(line):
    x1,y1,x2,y2 = line[0]
    return np.sqrt((x2-x1)**2+(y2-y1)**2)
def find_intercept(line):
    x1,y1,x2,y2 = line[0]
    return y1-find_slope(line)*x1

In [4]:
video_handler = cv.VideoCapture('./Data/whiteline.mp4')
if (video_handler.isOpened() == False):
    print("Error opening the video file")
else:
# Get frame rate information
    fps = int(video_handler.get(5))
    print("Frame Rate : ",fps,"frames per second")	
    # Get frame count
    frame_count = video_handler.get(7)
    print("Frame count : ", frame_count)

frame_width = int(video_handler.get(3))
frame_height = int(video_handler.get(4))
   
size = (frame_width, frame_height)
result = cv.VideoWriter('Question_2_result.avi', 
                         cv.VideoWriter_fourcc(*'MJPG'),
                         10, size)

Frame Rate :  25 frames per second
Frame count :  221.0


In [5]:
frames = []
i = 0
previous_tag=None
while(video_handler.isOpened()):
    # nonoise_vid.read() methods returns a tuple, first element is a bool 
    # and the second is frame
    ret, frame = video_handler.read()
    if ret == True:
        frames.append(frame)
        
        ## Convert Blur image and apply adaptive histogram equalization
        blurred = cv.GaussianBlur(frame,(5,5),0)
        l,a,b = cv.split(cv.cvtColor(blurred,cv.COLOR_BGR2LAB))
        clahe = cv.createCLAHE(clipLimit=2.0,tileGridSize=(8,8))
        adpt_hist = clahe.apply(l)
        corrected_image = cv.cvtColor((cv.merge((adpt_hist,a,b))),cv.COLOR_LAB2BGR)
        
        ## conver image to gray scale
        gray = cv.cvtColor(corrected_image,cv.COLOR_BGR2GRAY)
        ## apply thresholding
        ret, thresh = cv.threshold(gray,127,256,cv.THRESH_BINARY+cv.THRESH_OTSU)
        ## Apply canny edge dector
        edges_image = cv.Canny(thresh,50,150,apertureSize = 3)
        # showimage(thresh)
        
        ## Define the polygon that contains the area of interest
        ROI_points = np.array([[(150,frame.shape[0]),(450,310),(490,310),(880,frame.shape[0])]])
        masked_image = np.zeros_like(edges_image)
        mask = cv.fillPoly(masked_image,ROI_points,255)
        ## Apply mask to edges image
        masked_image = cv.bitwise_and(edges_image,mask)
        # showimage(masked_image)
        
        ## Apply Hough transform to find the lines
        lines = cv.HoughLinesP(masked_image,4,np.pi/180,100,np.array([]),minLineLength=70,maxLineGap=10)
        print(i)
        frame_cpy = np.copy(frame)
        right_line = []
        left_line = []
        draw_lines = []
        
        ## Find the slope and intercept of the lines
        try:
            for line in lines:
                if find_slope(line) < 0:
                    left_line.append(line)
                else:
                    right_line.append(line)
        except:
            pass
        
        ## Find the average length of each set of lines
        try:
            rl_avg_length = sum([length_of_line(x) for x in right_line])/len(right_line)
        except:
            rl_avg_length = 0
        try:
            ll_avg_length = sum([length_of_line(x) for x in left_line])/len(left_line)
        except:
            ll_avg_length = 0
        solid_lines = []
        dashed_lines = []
        
        ## Find the lines that are solid or dashed
        if(rl_avg_length > ll_avg_length):
            solid_lines = right_line
            dashed_lines = left_line
        else:
            solid_lines = left_line
            dashed_lines = right_line
        lines_of_interest = []
        
        ## In the event that the no line was detected, the previous line is used
        try:
            lines_of_interest.append(dashed_lines[0])
        except:
            lines_of_interest.append(previous_line[0])
        try:
            lines_of_interest.append(solid_lines[0])
        except:
            lines_of_interest.append(previous_line[1])
            
        ## Find the endpoints of the line, and draw the line
        for line in lines_of_interest:
            inter = find_intercept(line)
            start_point = (frame.shape[0]-inter)/find_slope(line)
            start_point = (int(start_point),frame.shape[0])
            endpoint = (350-inter)/find_slope(line)
            endpoint = (int(endpoint),350)
            draw_lines.append([start_point,endpoint])
        frame_cpy = np.copy(frame)
        previous_line = lines_of_interest 
        cv.line(frame_cpy, draw_lines[0][0], draw_lines[0][1], (0,0,255), 3, cv.LINE_AA)
        cv.line(frame_cpy, draw_lines[1][0], draw_lines[1][1], (0,255,0), 3, cv.LINE_AA)
        cv.imshow('Frame', frame_cpy)
        
        ## Write the frame to the output video
        if cv.waitKey(0) & 0xFF == ord('s'):
            cv.destroyAllWindows()
            break
        result.write(frame_cpy)
        i += 1
    else:
        cv.waitKey()
        cv.destroyAllWindows()
        break

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
